In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import requests
import scipy.stats as stats
import numpy as np

import plotly.express as px
import plotly.graph_objs as go
from sklearn.preprocessing import MinMaxScaler

from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, LabelEncoder, StandardScaler
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold

In [3]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold

In [4]:
# Imported this dataframe that was created in our EDA notebook
flights_df = pd.read_csv("/Users/ckz/Desktop/DATA_SCIENCE_BOOTCAMP/Midterm-project/flights.csv")

# print the first few rows of the DataFrame
flights_df.head()

/var/folders/j1/j5f5974x3sj1crjx533p6k9h0000gn/T/ipykernel_45830/2849657743.py:2: DtypeWarning: Columns (25) have mixed types. Specify dtype option on import or set low_memory=False.
  flights_df = pd.read_csv("/Users/ckz/Desktop/DATA_SCIENCE_BOOTCAMP/Midterm-project/flights.csv")


,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,...,distance,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,first_dep_time,total_add_gtime,longest_add_gtime,no_name
0,2018-01-01,B6,B6,B6,880,B6,N794JB,880,12954,LGB,...,231.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2018-01-01,B6,B6,B6,885,B6,N337JB,885,12478,JFK,...,427.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2018-01-01,B6,B6,B6,886,B6,N337JB,886,14492,RDU,...,427.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2018-01-01,B6,B6,B6,889,B6,N318JB,889,11278,DCA,...,444.0,0.0,0.0,8.0,0.0,10.0,NaN,NaN,NaN,NaN
4,2018-01-01,B6,B6,B6,891,B6,N348JB,891,10721,BOS,...,1185.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# # Shuffle the data in the DataFrame and create a sample with half the size
sample_size = len(flights_df) // 10

flights_df_sample = flights_df.sample(n=sample_size, random_state=42)

## Reset the index of the shuffled sample

flights_df_sample.reset_index(drop=True, inplace=True)

In [6]:
flights_test = pd.read_csv("/Users/ckz/Desktop/DATA_SCIENCE_BOOTCAMP/Midterm-project/flight_test.csv")

In [7]:
flights_df_sample.columns

Index(['fl_date', 'mkt_unique_carrier', 'branded_code_share', 'mkt_carrier',
       'mkt_carrier_fl_num', 'op_unique_carrier', 'tail_num',
       'op_carrier_fl_num', 'origin_airport_id', 'origin', 'origin_city_name',
       'dest_airport_id', 'dest', 'dest_city_name', 'crs_dep_time', 'dep_time',
       'dep_delay', 'taxi_out', 'wheels_off', 'wheels_on', 'taxi_in',
       'crs_arr_time', 'arr_time', 'arr_delay', 'cancelled',
       'cancellation_code', 'diverted', 'dup', 'crs_elapsed_time',
       'actual_elapsed_time', 'air_time', 'flights', 'distance',
       'carrier_delay', 'weather_delay', 'nas_delay', 'security_delay',
       'late_aircraft_delay', 'first_dep_time', 'total_add_gtime',
       'longest_add_gtime', 'no_name'],
      dtype='object')

In [8]:
flights_test.columns

Index(['fl_date', 'mkt_unique_carrier', 'branded_code_share', 'mkt_carrier',
       'mkt_carrier_fl_num', 'op_unique_carrier', 'tail_num',
       'op_carrier_fl_num', 'origin_airport_id', 'origin', 'origin_city_name',
       'dest_airport_id', 'dest', 'dest_city_name', 'crs_dep_time',
       'crs_arr_time', 'dup', 'crs_elapsed_time', 'flights', 'distance'],
      dtype='object')

In [9]:
flights_df_sample = flights_df_sample[['fl_date','arr_delay','mkt_unique_carrier', 'branded_code_share', 'mkt_carrier',
       'mkt_carrier_fl_num', 'op_unique_carrier', 'tail_num',
       'op_carrier_fl_num', 'origin_airport_id', 'origin', 'origin_city_name',
       'dest_airport_id', 'dest', 'dest_city_name', 'crs_dep_time',
       'crs_arr_time', 'dup', 'crs_elapsed_time', 'flights', 'distance']].copy()

In [10]:
# Extract month from fl_date column
flights_df_sample['month'] = pd.to_datetime(flights_df_sample['fl_date']).dt.month

In [11]:
flights_test['month'] = pd.to_datetime(flights_test['fl_date']).dt.month

In [12]:
flights_df_sample['avg_monthly_arr_delay'] = flights_df_sample.groupby('month')['arr_delay'].transform('mean')

In [13]:
value_counts = flights_df_sample['avg_monthly_arr_delay'].value_counts()
print(value_counts)

9.209757    14129
9.396488    14122
2.961378    13790
2.866672    13757
7.073326    13626
9.866914    13517
4.676415    13400
2.231817    13134
1.693996    12996
3.761830    12988
3.630774    12378
7.233239    11272
Name: avg_monthly_arr_delay, dtype: int64


In [14]:
import pandas as pd

# Create a dictionary with month as keys and corresponding average delay as values
average_delays = {
    1: 9.209757,
    2: 9.396488,
    3: 2.961378,
    4: 2.866672,
    5: 7.073326,
    6: 9.866914,
    7: 4.676415,
    8: 2.231817,
    9: 1.693996,
    10: 3.761830,
    11: 3.630774,
    12: 7.233239
}

# Map the average delays based on the 'month' column
flights_test['avg_monthly_arr_delay'] = flights_test['month'].map(average_delays)

In [15]:
flights_test['avg_monthly_arr_delay'].isnull().sum()

0

In [25]:
flights_df_s = flights_df_sample[['fl_date','arr_delay','mkt_unique_carrier', 'distance','avg_monthly_arr_delay','month']].copy()

In [26]:
flights_t = flights_test[['fl_date','mkt_unique_carrier', 'distance','avg_monthly_arr_delay','month']].copy()

In [27]:
df_encoded = pd.get_dummies(data=flights_df_s, columns=['mkt_unique_carrier','month'] , drop_first=False)



In [21]:
df_encoded.head()

,fl_date,arr_delay,distance,avg_monthly_arr_delay,mkt_unique_carrier_AA,mkt_unique_carrier_AS,mkt_unique_carrier_B6,mkt_unique_carrier_DL,mkt_unique_carrier_F9,mkt_unique_carrier_G4,mkt_unique_carrier_HA,mkt_unique_carrier_NK,mkt_unique_carrier_UA,mkt_unique_carrier_VX,mkt_unique_carrier_WN
0,2019-11-02,9.0,446.0,2.231817,0,0,0,0,0,0,0,0,0,0,1
1,2018-01-29,-13.0,696.0,3.630774,0,0,0,0,0,0,0,0,0,0,1
2,2018-03-29,7.0,294.0,2.961378,1,0,0,0,0,0,0,0,0,0,0
3,2019-03-27,-8.0,328.0,2.961378,1,0,0,0,0,0,0,0,0,0,0
4,2019-06-26,-6.0,451.0,9.866914,0,0,0,0,0,0,0,0,1,0,0


In [28]:
df_encoded.columns

Index(['fl_date', 'arr_delay', 'distance', 'avg_monthly_arr_delay',
       'mkt_unique_carrier_AA', 'mkt_unique_carrier_AS',
       'mkt_unique_carrier_B6', 'mkt_unique_carrier_DL',
       'mkt_unique_carrier_F9', 'mkt_unique_carrier_G4',
       'mkt_unique_carrier_HA', 'mkt_unique_carrier_NK',
       'mkt_unique_carrier_UA', 'mkt_unique_carrier_VX',
       'mkt_unique_carrier_WN', 'month_1', 'month_2', 'month_3', 'month_4',
       'month_5', 'month_6', 'month_7', 'month_8', 'month_9', 'month_10',
       'month_11', 'month_12'],
      dtype='object')

In [35]:
test_encoded = pd.get_dummies(data=flights_t, columns=['mkt_unique_carrier','month'] , drop_first=False)

In [32]:
df_encoded['arr_delay'].isnull().sum()

3087

In [33]:
df_encoded = df_encoded.dropna(subset=['arr_delay'])

In [37]:
test_encoded.columns

Index(['fl_date', 'distance', 'avg_monthly_arr_delay', 'mkt_unique_carrier_AA',
       'mkt_unique_carrier_AS', 'mkt_unique_carrier_B6',
       'mkt_unique_carrier_DL', 'mkt_unique_carrier_F9',
       'mkt_unique_carrier_G4', 'mkt_unique_carrier_HA',
       'mkt_unique_carrier_NK', 'mkt_unique_carrier_UA',
       'mkt_unique_carrier_WN', 'month_1'],
      dtype='object')

In [40]:
# Linear regression model

# Separate the input features (X) and the target variable (y)
X = df_encoded[['distance', 'avg_monthly_arr_delay',
       'mkt_unique_carrier_AA', 'mkt_unique_carrier_AS',
       'mkt_unique_carrier_B6', 'mkt_unique_carrier_DL',
       'mkt_unique_carrier_F9', 'mkt_unique_carrier_G4',
       'mkt_unique_carrier_HA', 'mkt_unique_carrier_NK',
       'mkt_unique_carrier_UA', 
       'mkt_unique_carrier_WN','month_1']]

y = df_encoded['arr_delay']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a linear regression model
model = LinearRegression()

# Train the model using the training data
model.fit(X_train, y_train)

# Make predictions on the testing data
y_pred = model.predict(X_test)

# Evaluate the model
score = model.score(X_test, y_test)

# Print the coefficient and intercept of the linear regression model
print('Coefficients:', model.coef_)
print('Intercept:', model.intercept_)
print('R^2 Score:', score)

Coefficients: [-1.85659984e-03  9.75977791e-01 -7.18208326e-01 -5.17642874e+00
  4.87726007e+00 -4.10733195e+00  5.91921991e+00  3.12206997e+00
 -5.19253640e+00 -2.31013607e-01  3.53374732e+00 -3.15079460e+00
 -3.05867032e-01]
Intercept: 2.5320018091028795
R^2 Score: 0.00846028358015527


In [41]:
X_new = test_encoded[['distance', 'avg_monthly_arr_delay',
                'mkt_unique_carrier_AA', 'mkt_unique_carrier_AS',
                'mkt_unique_carrier_B6', 'mkt_unique_carrier_DL',
                'mkt_unique_carrier_F9', 'mkt_unique_carrier_G4',
                'mkt_unique_carrier_HA', 'mkt_unique_carrier_NK',
                'mkt_unique_carrier_UA', 
                'mkt_unique_carrier_WN', 'month_1']]

# Make predictions on the new dataset
y_pred_new = model.predict(X_new)

In [42]:
flights_test['predicted_delays'] = y_pred_new

In [43]:
flights_test.columns

Index(['fl_date', 'mkt_unique_carrier', 'branded_code_share', 'mkt_carrier',
       'mkt_carrier_fl_num', 'op_unique_carrier', 'tail_num',
       'op_carrier_fl_num', 'origin_airport_id', 'origin', 'origin_city_name',
       'dest_airport_id', 'dest', 'dest_city_name', 'crs_dep_time',
       'crs_arr_time', 'dup', 'crs_elapsed_time', 'flights', 'distance',
       'month', 'avg_monthly_arr_delay', 'predicted_delays'],
      dtype='object')

In [44]:
flights_test['predicted_delays'].head(20)

0     4.961480
1     7.419618
2     7.419618
3     4.402644
4     6.170127
5     6.170127
6     6.170127
7     6.170127
8     6.170127
9     6.170127
10    6.448617
11    5.301238
12    5.301238
13    7.590426
14    7.590426
15    7.590426
16    7.590426
17    7.590426
18    7.590426
19    7.590426
Name: predicted_delays, dtype: float64

In [46]:
Predicted_delay = flights_test[['fl_date','mkt_carrier','mkt_carrier_fl_num','origin','dest','predicted_delays']].copy()

In [47]:
Predicted_delay.to_csv('Predicted_delays.csv', index=False)

In [48]:
predicted_delay_sample = Predicted_delay.sample(n=100, random_state=42)

In [49]:
predicted_delay_sample.to_csv('predicted_delay_sample.csv', index=False)

In [51]:
start_date = '2020-01-01'
end_date = '2020-01-07'

mask = (Predicted_delay['fl_date'] >= start_date) & (Predicted_delay['fl_date'] <= end_date)
random_sample = Predicted_delay.loc[mask].sample(n=100, random_state=42)

In [52]:
random_sample.to_csv('prediction_sample.csv', index=False)

In [53]:
Predicted_delay.describe()

,mkt_carrier_fl_num,predicted_delays
count,67958.000000,67958.000000
mean,2767.723947,8.825413
std,1848.512759,3.338904
min,1.000000,-3.437260
25%,1162.000000,6.277421
50%,2378.000000,8.346502
75%,4468.000000,10.938658
max,6860.000000,16.695715
